# Analyze results for 3D CGAN
Feb 22, 2021

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/5_3d_cgan/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


### Read validation data

In [7]:
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use

bins=f_transform(bins)   ### scale to (-1,1)
# ### Extract validation data
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=range(len(sigma_lst))
bkgnd_dict={}
num_bkgnd=200

for label in labels_lst:
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:]
    
    dict_val=post.f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
    del samples

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_1.1_train_val.npy


## Read data

In [8]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [9]:
dict1={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [10]:
# parent_dir=u.result
img_size=64
parent_dir=dict1[str(img_size)]
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'202106*')]
n=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(n)

interactive(children=(Dropdown(description='x', options=('20210620_113852_cgan_bs32_nodes8_lr0.0001-vary_fm50_…

In [72]:
result=n.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210619_224213_cgan_bs32_nodes8_lr0.0001-vary_fm50


## Plot Losses

In [73]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [74]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
37431,37431.0,63.0,0.235034,0.232714,0.467747,3.255831,13.668125,10.399620,-3.108395,10.810431,-2.597854,NaN,0.012675,2.987896,-2.858304,-3.189602,0.793974,0.000006,0.000006
37432,37432.0,63.0,0.181642,0.143099,0.324741,2.814492,13.428387,10.566786,-2.846118,10.882505,-2.873352,NaN,0.047109,2.161906,-2.859251,-2.713832,0.786009,0.000006,0.000006
37433,37433.0,63.0,0.112397,0.166939,0.279336,3.196909,13.591529,10.370832,-2.994292,10.839382,-2.746268,NaN,0.023787,3.720149,-2.854005,-3.125091,0.790214,0.000006,0.000006
37434,37434.0,63.0,0.130117,0.158906,0.289023,3.211007,13.767612,10.552769,-3.113134,10.865761,-2.736638,NaN,0.003836,3.377064,-2.684884,-3.152395,0.787756,0.000006,0.000006
37435,37435.0,63.0,0.208673,0.116374,0.325048,3.026676,13.570823,10.520730,-2.825980,10.885996,-2.769790,NaN,0.023417,2.464508,-3.265838,-2.945090,0.789535,0.000006,0.000006
37436,37436.0,63.0,0.206783,0.123580,0.330362,2.933946,13.425792,10.482430,-3.005218,10.812387,-2.672103,NaN,0.009415,2.741296,-2.859067,-2.851160,0.786281,0.000006,0.000006
37437,37437.0,63.0,0.187609,0.188728,0.376337,2.986371,13.622642,10.491955,-3.132936,10.798526,-2.794070,NaN,0.144316,2.978965,-2.376906,-2.913350,0.786684,0.000006,0.000006
37438,37438.0,63.0,0.210208,0.199168,0.409376,3.191043,13.541048,10.342420,-2.963267,10.805973,-2.967496,NaN,0.007586,2.843654,-2.967982,-3.130363,0.787575,0.000006,0.000006
37439,37439.0,63.0,0.235315,0.204188,0.439503,2.998589,13.780254,10.744900,-2.622908,10.799562,-2.869864,NaN,0.036766,2.398518,-3.589211,-2.925466,0.789683,0.000006,0.000006
37440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.852632,-2.517478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [76]:

chi=df_metrics.quantile(q=0.2,axis=0)['hist_chi']
print(chi)
df_metrics[(df_metrics['hist_chi']<=chi)&(df_metrics.epoch>1)].sort_values(by=['hist_chi']).head(10)

-2.9145259857177734


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
15046,15046.0,25.0,0.230558,0.207576,0.438134,2.932430,13.527962,10.582875,-2.997287,10.673954,-3.238003,NaN,0.012656,2.078210,-3.272564,-2.842322,0.787944,0.000050,0.000050
15749,15749.0,26.0,0.161982,1.013336,1.175318,8.525791,19.441900,10.719789,-3.100010,10.729021,-3.236508,NaN,0.196322,5.998387,0.219498,-8.525433,0.787826,0.000050,0.000050
17557,17557.0,30.0,0.117966,0.191804,0.309770,3.115794,13.783837,10.627157,-3.202205,10.683758,-3.235552,NaN,0.040886,3.053597,-2.703843,-3.040881,0.790611,0.000025,0.000025
17207,17207.0,29.0,0.224839,0.159016,0.383855,3.134042,13.814713,10.586383,-2.556893,10.737841,-3.232087,NaN,0.094289,2.060948,-3.544134,-3.040227,0.787281,0.000050,0.000050
13130,13130.0,22.0,0.160659,0.125126,0.285785,3.334771,14.020349,10.643251,-2.570519,10.700977,-3.229923,NaN,0.042326,2.780260,-3.333513,-3.225606,0.795081,0.000050,0.000050
13208,13208.0,22.0,0.267253,0.137652,0.404905,3.583318,14.988206,11.203830,-2.619559,10.568878,-3.229175,NaN,0.201058,2.011998,-4.088627,-3.534831,0.784907,0.000050,0.000050
15521,15521.0,26.0,0.205813,0.140176,0.345989,3.473957,14.028960,10.544765,-2.979441,10.745359,-3.229048,NaN,0.010238,2.592885,-2.814688,-3.388710,0.788644,0.000050,0.000050
15236,15236.0,26.0,0.145862,0.222336,0.368198,4.612967,15.167688,10.389866,-2.874555,10.754705,-3.228173,NaN,0.164855,3.635245,-1.934153,-4.586347,0.796990,0.000050,0.000050
14638,14638.0,25.0,0.112755,0.224206,0.336961,3.313110,13.808804,10.482378,-2.958272,10.972198,-3.225848,NaN,0.013315,3.609493,-2.031829,-3.247351,0.785878,0.000050,0.000050
15358,15358.0,26.0,0.131761,0.250865,0.382625,4.488227,15.139226,10.567974,-2.365833,10.749254,-3.225713,NaN,0.083025,3.368795,-1.847109,-4.468691,0.786973,0.000050,0.000050


In [77]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

<!-- ### Read validation data -->

## Read stored chi-squares for images

In [78]:
## Get sigma list from saved files
flist=glob.glob(result_dir+'/df_processed*')
sigma_lst=[i.split('/')[-1].split('df_processed_')[-1].split('.pkle')[0] for i in flist]
sigma_lst.sort() ### Sorting is important for labels to match !!

labels_lst=np.arange(len(sigma_lst))

In [79]:
sigma_lst,labels_lst

(['0.5', '0.8', '1.1'], array([0, 1, 2]))

In [80]:
### Create a merged dataframe

df_list=[]
for label in labels_lst:
    df=pd.read_pickle(result_dir+'/df_processed_{0}.pkle'.format(str(sigma_lst[label])))
    df[['epoch','step']]=df[['epoch','step']].astype(int)
    df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting
    df_list.append(df)

for i,df in enumerate(df_list):
    df1=df.add_suffix('_'+str(i))
    # renaming the columns to be joined on
    keys=['epoch','step','img_type','label']
    rename_cols_dict={key+'_'+str(i):key for key in keys}
#     print(rename_cols_dict)
    df1.rename(columns=rename_cols_dict,inplace=True) 
    df_list[i]=df1
    
df_merged=reduce(lambda x, y : pd.merge(x, y, on = ['step','epoch','img_type','label']), df_list)

### Get sum of all 4 classes for 3 types of chi-squares
for chi_type in ['chi_1','chi_spec1','chi_1b']:
    keys=[chi_type+'_'+str(label) for label in labels_lst]
#     display(df_merged[keys].sum(axis=1))
    df_merged['sum_'+chi_type]=df_merged[keys].sum(axis=1)
del df_list


### Slice best steps

In [81]:
def f_slice_merged_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',label='all',params_lst=[0,1,2],head=10,epoch_range=[0,None],use_sum=True,display_flag=False):
    ''' View dataframe after slicing
    '''

    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ######### Apply cutoff to keep reasonable chi1 and chispec1
    #### Add chi-square columns to use
    chi_cols=[]
    if use_sum: ## Add sum chi-square columns
        for j in ['chi_1','chi_spec1','chi_1b']: chi_cols.append('sum_'+j)
        
    if label=='all': ### Add chi-squares for all labels
        for j in ['chi_1','chi_spec1','chi_1b']:
            for idx,i in enumerate(params_lst): chi_cols.append(j+'_'+str(idx))
    else: ## Add chi-square for specific label
        assert label in params_lst, "label %s is not in %s"%(label,params_lst)
        label_idx=params_lst.index(label)
        print(label_idx)
        for j in ['chi_1','chi_spec1','chi_1b']: chi_cols.append(j+'_'+str(label_idx))
#     print(chi_cols)
    q_dict=dict(df_merged.quantile(q=cutoff,axis=0)[chi_cols])
    # print(q_dict)
    strg=['%s < %s'%(key,q_dict[key]) for key in chi_cols ]
    query=" & ".join(strg)
    # print(query)
    df=df.query(query)
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
    chis=[i for i in df_merged.columns if 'chi' in i]
    col_list=['label']+chis+['epoch','step']
    if (col_mode=='short'): 
        col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
        col_list=['label']+chi_cols
    df2=df1.head(head)[col_list]
    
    if display_flag: display(df2) # Display df
    
    return df2

# f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_1',label=0.65,params_lst=[0.5,0.65,0.8,1.1],use_sum=True,head=2000,display_flag=False,epoch_range=[7,None])

In [82]:
cols_to_sort=np.unique([i for i in df_merged.columns for j in ['chi_1_','chi_spec1_'] if ((i.startswith(j)) or (i.startswith('sum')))])

w=interactive(f_slice_merged_df,df=fixed(df_merged),
cutoff=widgets.FloatSlider(value=0.3, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
use_sum=widgets.Checkbox(value=True),
label=ToggleButtons(options=['all']+sigma_lst), params_lst=fixed(sigma_lst),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=cols_to_sort
)
display(w)

interactive(children=(FloatSlider(value=0.3, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [83]:
df_sliced=w.result
[int(i.split('-')[1]) for i in df_sliced.label.values]

# df_sliced

[]

In [101]:
best_step=[]
df_test=df_merged.copy()
df_test=df_merged[(df_merged.epoch<5000)&(df_merged.epoch>50)]
cut_off=1.0

best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_spec1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_1b',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)

# best_step.append([46669,34281])
best_step=np.unique([i for j in best_step for i in j])
print(best_step)
best_step

[32220 32640 32730 33080 34380 34400 34460 37810]


array([32220, 32640, 32730, 33080, 34380, 34400, 34460, 37810])

In [102]:
# best_step=[6176]
# best_step= [23985,24570,25155,25740,26325,26910,27495]
# best_step=[int(i.split('-')[1]) for i in df_sliced.label.values]
# best_step=np.arange(40130,40135).astype(int)

In [103]:
df_best=df_merged[df_merged.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(8, 61)
[(55, 32220), (55, 32640), (55, 32730), (56, 33080), (58, 34380), (58, 34400), (58, 34460), (64, 37810)]


In [104]:
col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
df_best[col_list]

,label,sum_chi_1,sum_chi_spec1,sum_chi_1b
3221,55-32220,0.771336,1.126871,0.687382
3263,55-32640,0.688749,1.400893,0.547141
3272,55-32730,0.553809,1.545479,0.478267
3307,56-33080,0.710590,1.557728,0.608751
3437,58-34380,2.024703,0.794535,1.893098
3439,58-34400,2.833897,0.800463,2.655702
3445,58-34460,3.070578,0.810841,2.931690
3780,64-37810,1.381727,0.800364,1.306743


### Interactive plot

In [105]:

def f_plot_hist_spec(df,param_labels,sigma_lst,steps_list,bkg_dict,plot_type,img_size):

    assert plot_type in ['hist','spec','grid','spec_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    for par_label in param_labels:
        df=df[df.step.isin(steps_list)]
#         print(df.shape)
        idx=sigma_lst.index(par_label)
        suffix='_%s'%(idx)
        dict_bkg=bkg_dict[str(par_label)]
        
        for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
            label=row.label+'_'+str(par_label)
            if plot_type=='hist':
                x1=row['hist_bin_centers'+suffix]
                y1=row['hist_val'+suffix]
                yerr1=row['hist_err'+suffix]
                x1=f_invtransform(x1)
                
                plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            if plot_type=='spec':
                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]/np.sqrt(row['num_imgs'+suffix])
                x2=np.arange(len(y2))

                plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
                plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

            if plot_type=='spec_relative':

                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]
                x2=np.arange(len(y2))

                ### Reference spectrum
                y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
                y=y2/y1
                ## Variance is sum of variance of both variables, since they are uncorrelated

                # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
                yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'+suffix])
                
                plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
                plt.plot(x2, y, marker=marker, linestyle=':',label=label)

            if plot_type=='grid':
                images=np.load(row['fname'+suffix])[:,0,:,:,0]
                print(images.shape)
                f_plot_grid(images[:8],cols=4,fig_size=(8,4))
            
        ### Plot reference data
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)
        
        if plot_type=='spec':
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            x=np.arange(len(y))
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
            plt.xlim(0,img_size/2)

        if plot_type=='spec_relative':
            plt.axhline(y=1.0,color='k',linestyle='-.')
            plt.title("Relative spectrum")
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)

    if plot_type in ['hist','spec']:     
        plt.yscale('log')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})


# f_plot_hist_spec(df_merged,[sigma_lst[-1]],sigma_lst,[best_step[0]],bkgnd_dict,'hist')

In [106]:
interact_manual(f_plot_hist_spec,df=fixed(df_best),
                param_labels=SelectMultiple(options=sigma_lst),sigma_lst=fixed(sigma_lst),
                steps_list=SelectMultiple(options=df_best.step.values),
                img_size=fixed(img_size),
                bkg_dict=fixed(bkgnd_dict),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))

interactive(children=(SelectMultiple(description='param_labels', options=('0.5', '0.8', '1.1'), value=()), Sel…

<function __main__.f_plot_hist_spec(df, param_labels, sigma_lst, steps_list, bkg_dict, plot_type, img_size)>

In [65]:
best_step

array([30180, 32110, 32530, 32540, 32590, 32600, 32940, 33380])

In [ ]:
sigma=0.5
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_{0}_train_val.npy'.format(sigma)
a1=np.load(fname,mmap_mode='r')[-100:]
a1.shape

images=a1[:,0,:,:,0]
f_plot_grid(images[:8],cols=4,fig_size=(8,4))


### Delete unwanted stored models
(Since deterministic runs aren't working well )

In [ ]:
# # fldr='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210119_134802_cgan_predict_0.65_m2/models'
# fldr=result_dir
# print(fldr)
# flist=glob.glob(fldr+'/models/checkpoint_*.tar')
# len(flist)

In [ ]:
# # Delete unwanted stored images
# for i in flist:
#     try:
#         step=int(i.split('/')[-1].split('_')[-1].split('.')[0])
#         if step not in best_step:
# #             print("Found")
# #             print(step)
#             os.remove(i)
#             pass
#         else: 
#             print(step)
# #             print(i)
#     except Exception as e:
# #         print(e)
# #         print(i)
#         pass

In [ ]:
best_step

In [ ]:
# ! du -hs  /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy